# PyLossless pipeline: A step-by-step demonstration

This Notebook details each pipeline step. First, the mathematical notation for the operation will be described. Second, the python code for the operation will be shown. We will go through the first step in details. Similar procedures are used in all following steps so for the sake of brevity we will increasingly explain less of the formulas throughout this tutorial

# ----------------------------------------------------------------------------

# Cloning and installing pylossless

In [ ]:
# THIS CODE ONLY NEEDS TO BE RUN ONCE.
# IF PYLOSSLESS IS ALREADY AVAILABLE, IT DOES NOT NEED TO BE RERUN.
try:
  import pylossless
except ModuleNotFoundError:
  !git clone https://github.com/lina-usc/pylossless.git
  %pip install --quiet --editable ./pylossless

  # RESTARTING THE KERNEL AFTER THE PIP INSTALL
  # THIS IS NECESSARY FOR THE NEWLY INSTALLED PACKAGE
  # TO BE CORRECTLY IMPORTED.
  exit()

Cloning into 'pylossless'...
remote: Enumerating objects: 2278, done.
remote: Counting objects: 100% (2278/2278), done.
remote: Compressing objects: 100% (827/827), done.
remote: Total 2278 (delta 1401), reused 2195 (delta 1374), pack-reused 0
Receiving objects: 100% (2278/2278), 216.46 MiB | 25.05 MiB/s, done.
Resolving deltas: 100% (1401/1401), done.
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 91.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 90.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.6/21.6 MB 42.5 MB/s eta 0:00:00
  Building editable for pylossless (pyproject.toml) ... done



<br>

--------------------------------
# Getting a LosslessPipeline object

 Create a pipeline object and manually set a raw object to it. We proceed this way to be able to demonstrate the operation for each step, one at a time. Normally, one would simply load and process a raw object with `pipeline.run_with_raw(raw)`. This demonstration relies on test datasets from MNE. It may take a while to download the MNE sample data (about 2-3 minutes on Google Colab). **Please be patient**...

In [ ]:
from pathlib import Path

import numpy as np

import mne
from mne.datasets import sample

import pylossless as ll

# LOAD DATA FROM MNE FOR THE DEMONSTRATION
data_path = sample.data_path()
meg_path = data_path / 'MEG' / 'sample'
raw_fname = meg_path / 'sample_audvis_raw.fif'
raw = mne.io.read_raw_fif(raw_fname, preload=True)

# LOAD DEFAULT CONFIG
config = ll.config.Config()
config.load_default()
config.save("test_config.yaml")

# CREATE A PIPELINE AN MANUALLY SET A RAW OBJECT
pipeline = ll.LosslessPipeline('test_config.yaml')
pipeline.raw = raw

Using default location ~/mne_data for sample...
Creating ~/mne_data


100%|██████████████████████████████████████| 1.65G/1.65G [00:00<00:00, 462GB/s]
Untarring contents of '/root/mne_data/MNE-sample-data-processed.tar.gz' to '/root/mne_data'


Attempting to create new mne-python configuration file:
/root/.mne/mne-python.json
Download complete in 01m11s (1576.2 MB)
Opening raw data file /root/mne_data/MNE-sample-data/MEG/sample/sample_audvis_raw.fif...
    Read a total of 3 projection items:
        PCA-v1 (1 x 102)  idle
        PCA-v2 (1 x 102)  idle
        PCA-v3 (1 x 102)  idle
    Range : 25800 ... 192599 =     42.956 ...   320.670 secs
Ready.
Reading 0 ... 166799  =      0.000 ...   277.714 secs...


--------------------------

# Preamble: Notation

We define:
- s, e, and t, as sensor, epochs, and time dimensions, respectively.

- a 3D matrix $X \in \mathbb{R}^{S_\mathcal{G} \times E_\mathcal{G} \times T}$, where $S_\mathcal{G}$  and $E_\mathcal{G}$ are the set of good sensors and epochs respectively, and $T$ is the number of time points (i.e. samples)

- We use superscript to denote operations across a dimension, and we use subscript to denote indexing a dimension


- a single sensor $i$ as $ X\big|_{s=i} \in \mathbb{R}^{E_\mathcal{G} \times T}$, with $i \in S_\mathcal{G}$

- a single epoch $j$ as  $X\big|_{e=j} \in \mathbb{R}^{S_\mathcal{G} \times T}$, with $j \in E_\mathcal{G} $

- sensor specific thresholds for rejecting epochs as $\tau^e_i \in \mathbb{R}^{S_\mathcal{G}}$

- epoch specific thresholds for rejecting sensors as $\tau^s_j \in \mathbb{R}^{E_\mathcal{G}}$


- *quantiles* as $Q\#^{dim}$: i.e. $Q75^s$ is the 75th *quantile* along the sensor dimension. The function $Q75^s(X)$ computes the 75th quantile along the $s$ dimension of matrix $X$, resulting in a matrix noted $X^{Q75^s} \in \mathbb{R}^{E \times T}$

Throughout the text, we use capital letters for matrices and lower-case letters for scalars. For example, the data point for sensor $i$, epoch $j$ and time $k$ as $X\big|_{s=i; e=j; t=k} = x_{ijk}\in \mathbb{R}$, and $X=\{x_{ijk}\}$.

<br>

------------------------

# Step 0: Input Data

### First, we epoch the data to be used for step 1

Let our 3D matrix below be defined as $X \in \mathbb{R}^{S \times E \times T}$ where $X$ is a matrix of real numbers and of dimension $S$ sensors $\times$ $E$ epochs $\times$ $T$ times.

In [ ]:
epochs = pipeline.get_epochs()

🧹 Epoching..
Not setting metadata
277 matching events found
No baseline correction applied
Created an SSP operator (subspace dimension = 3)
3 projection items activated
Using data from preloaded Raw for 277 events and 602 original time points ...
0 bad epochs dropped
Removing projector <Projection | PCA-v1, active : True, n_channels : 102>
Removing projector <Projection | PCA-v2, active : True, n_channels : 102>
Removing projector <Projection | PCA-v3, active : True, n_channels : 102>
🔍 Detecting channels to leave out of reference.
EEG channel type selected for re-referencing
Applying a custom ('EEG',) reference.


In [ ]:
# Let us convert our epochs array into a Named Array
from pylossless.pipeline import epochs_to_xr
epochs_xr = epochs_to_xr(epochs, kind='ch')
epochs_xr # 277 epochs, 50 sensors, 602 samples per epoch

<xarray.DataArray (epoch: 277, ch: 59, time: 602)>
array([[[-4.26965878e-05, -4.55733025e-05, -4.78312309e-05, ...,
         -8.13633825e-05, -7.19575301e-05, -5.53443722e-05],
        [-1.12694344e-05, -1.20099186e-05, -1.16296023e-05, ...,
         -1.14285380e-05, -1.30872915e-05, -1.57002080e-05],
        [-1.78352282e-05, -2.42311691e-05, -2.27521589e-05, ...,
         -9.11264817e-06, -5.92725481e-06, -5.81562304e-06],
        ...,
        [ 1.50136773e-05,  1.54548337e-05,  1.51369078e-05, ...,
          2.22599691e-05,  2.24156382e-05,  2.29380975e-05],
        [-1.52412965e-05, -1.46724240e-05, -1.45173275e-05, ...,
         -9.62922101e-06, -9.64063562e-06, -9.27229001e-06],
        [ 1.17435988e-05,  1.25791034e-05,  1.26648640e-05, ...,
          1.92420881e-05,  1.86432406e-05,  1.95474139e-05]],

       [[-7.19575301e-05, -5.53443722e-05, -4.86540536e-05, ...,
         -5.48918032e-05, -5.74360574e-05, -6.05930207e-05],
        [-1.30872915e-05, -1.57002080e-05, -1.71937885e-05, ...,
         -2.33215219e-05, -2.28248519e-05, -2.31918983e-05],
        [-5.92725481e-06, -5.81562304e-06, -6.09936458e-06, ...,
         -2.03989737e-05, -1.92719632e-05, -1.77687091e-05],
...
        [ 8.58634883e-06,  9.56373789e-06,  1.01537047e-05, ...,
          2.49608841e-05,  2.48504591e-05,  2.47239989e-05],
        [-1.87772370e-05, -1.81752141e-05, -1.82906611e-05, ...,
         -3.40169454e-06, -3.79578042e-06, -3.83465514e-06],
        [ 4.34385855e-06,  5.55232672e-06,  5.85976184e-06, ...,
          1.83717758e-05,  1.94134504e-05,  2.12006254e-05]],

       [[-3.84674103e-05, -3.62507308e-05, -3.51405898e-05, ...,
         -6.71273687e-05, -6.00922247e-05, -5.45881906e-05],
        [-1.70288598e-06,  2.03242703e-08,  1.64636495e-06, ...,
         -9.68092889e-06, -1.11533229e-05, -1.10496795e-05],
        [-8.89306783e-06, -7.38259569e-06, -4.71025977e-06, ...,
         -1.77999741e-05, -1.83879992e-05, -1.83700798e-05],
        ...,
        [ 2.48504591e-05,  2.47239989e-05,  2.32465546e-05, ...,
          2.20773417e-05,  2.27953468e-05,  2.40101352e-05],
        [-3.79578042e-06, -3.83465514e-06, -5.35863884e-06, ...,
         -4.95543344e-06, -4.01205597e-06, -2.60134472e-06],
        [ 1.94134504e-05,  2.12006254e-05,  2.04842284e-05, ...,
          8.55354110e-06,  8.66345301e-06,  9.70703780e-06]]])
Coordinates:
  * epoch    (epoch) int64 0 1 2 3 4 5 6 7 8 ... 269 270 271 272 273 274 275 276
  * ch       (ch) <U7 'EEG 001' 'EEG 002' 'EEG 003' ... 'EEG 059' 'EEG 060'
  * time     (time) float64 0.0 0.001665 0.00333 0.004995 ... 0.9973 0.999 1.001

-------------------------

# Step 1: Flag Noisy Sensors

<img src="https://raw.githubusercontent.com/scott-huberty/wip_pipeline-figures/main/Flag_noisy_sensors.png" alt="Flag noisy channels" width="50%" height="50%">


## step 1a) Take standard deviation across temporal dimension

#### We take the standard deviation of $X \in \mathbb{R}^{S \times E \times T}$ across the samples dimension time, resulting in a 2D matrix $X^{\sigma_{t}} \in \mathbb{R}^{S \times E}$


![std across samples](https://raw.githubusercontent.com/scott-huberty/wip_pipeline-figures/main/samples_std.png)


<br>

---------------------------------


In [ ]:
# Take standard deviation across time dimension
trim_ch_sd = epochs_xr.std('time')
# display a small subset
trim_ch_sd

<xarray.DataArray (epoch: 277, ch: 59)>
array([[2.09317397e-05, 2.07409853e-05, 2.37844983e-05, ...,
        1.02134102e-05, 1.03778298e-05, 1.04739898e-05],
       [9.05601832e-06, 5.93182784e-06, 6.40472565e-06, ...,
        3.96677776e-06, 4.06302433e-06, 5.16254333e-06],
       [1.68993123e-05, 1.71782691e-05, 1.99054677e-05, ...,
        6.62557257e-06, 7.21096841e-06, 6.88823234e-06],
       ...,
       [7.53660770e-06, 4.88489208e-06, 5.60202700e-06, ...,
        4.31913820e-06, 4.89151789e-06, 4.85804656e-06],
       [7.14090704e-06, 5.97215435e-06, 5.68533179e-06, ...,
        5.04430008e-06, 6.60599989e-06, 5.46551902e-06],
       [6.83655090e-06, 5.43000137e-06, 7.72497789e-06, ...,
        5.68676607e-06, 5.92370636e-06, 6.39738775e-06]])
Coordinates:
  * epoch    (epoch) int64 0 1 2 3 4 5 6 7 8 ... 269 270 271 272 273 274 275 276
  * ch       (ch) <U7 'EEG 001' 'EEG 002' 'EEG 003' ... 'EEG 059' 'EEG 060'

------------------

## Take the 50th and 75th quantile across dimension sensor of $X^{\sigma_{t}}$


This operation results in two 1D vectors of size $E$:

- $ X^{{\sigma}_t{Q50^s}} = Q50^s(X^{\sigma_{t}}) \in \mathbb{R}^{E} $
- $ X^{{\sigma}_t{Q75^s}} = Q75^s(X^{\sigma_{t}}) \in \mathbb{R}^{E} $



In [ ]:
# Step a
q50, q75 = trim_ch_sd.quantile([.5, .75], dim='ch')
q50 # a 1D array of median stDev values across channels for each epoch

<xarray.DataArray (epoch: 277)>
array([7.78344051e-06, 4.63338827e-06, 5.52387408e-06, 5.93894116e-06,
       4.50482308e-06, 5.07330689e-06, 5.31537338e-06, 3.99708834e-06,
       4.88286966e-06, 4.38871177e-06, 3.97986129e-06, 4.94447817e-06,
       5.30618150e-06, 4.45782960e-06, 9.49356136e-06, 5.01344977e-06,
       5.82589868e-06, 4.31768294e-06, 5.01208333e-06, 5.13707111e-06,
       5.34513821e-06, 4.99363797e-06, 5.69412439e-06, 5.64172840e-06,
       4.97205652e-06, 5.16427951e-06, 5.61903451e-06, 5.60619948e-06,
       5.14529692e-06, 4.66396815e-06, 7.69433250e-06, 6.09347058e-06,
       8.20504903e-06, 5.21227459e-06, 4.94718451e-06, 5.05901985e-06,
       4.26430824e-06, 4.62247762e-06, 4.51554946e-06, 5.04936640e-06,
       5.02583286e-06, 4.73926307e-06, 5.44792746e-06, 8.38977230e-06,
       5.84820203e-06, 9.54935786e-06, 6.69182717e-06, 8.86301957e-06,
       7.14951085e-06, 5.94278231e-06, 4.80585406e-06, 5.97484967e-06,
       5.82097907e-06, 4.65073509e-06, 4.32334964e-06, 5.22753234e-06,
       6.80880269e-06, 7.72593312e-06, 5.47700694e-06, 8.46974424e-06,
       1.00479200e-05, 6.38100867e-06, 6.51553377e-06, 5.60645223e-06,
       5.21389348e-06, 5.94272651e-06, 6.48358990e-06, 4.32474911e-06,
       4.49999709e-06, 6.82698753e-06, 7.07695883e-06, 7.03796639e-06,
       7.44606651e-06, 6.70011523e-06, 7.17141399e-06, 6.35638025e-06,
       5.94935923e-06, 5.16169893e-06, 5.14041764e-06, 5.65707113e-06,
...
       5.48387087e-06, 4.47989711e-06, 6.24399009e-06, 6.33494192e-06,
       5.05158967e-06, 5.27718457e-06, 5.34736974e-06, 6.24322036e-06,
       6.45541749e-06, 4.89990156e-06, 5.86109131e-06, 6.69140747e-06,
       4.72525041e-06, 4.46732548e-06, 4.47174852e-06, 4.88277891e-06,
       6.37442693e-06, 6.15274111e-06, 8.71180085e-06, 1.21688378e-05,
       9.12329404e-06, 7.97626361e-06, 9.03812211e-06, 8.39270439e-06,
       5.67426919e-06, 5.52479797e-06, 5.88732267e-06, 6.82388407e-06,
       5.40185440e-06, 5.17385780e-06, 4.48677373e-06, 6.89004885e-06,
       7.31490377e-06, 7.11201507e-06, 5.33004844e-06, 5.42627323e-06,
       5.01352125e-06, 5.97711818e-06, 5.52002139e-06, 4.64928508e-06,
       7.00591560e-06, 7.74013044e-06, 8.51921830e-06, 8.64318537e-06,
       6.30082584e-06, 4.86369480e-06, 5.61218642e-06, 9.38071416e-06,
       7.27590073e-06, 9.41810810e-06, 9.65615293e-06, 8.03326898e-06,
       8.19164393e-06, 9.00280175e-06, 7.93624326e-06, 7.24049013e-06,
       6.96934693e-06, 8.36637921e-06, 8.32461715e-06, 6.96333366e-06,
       6.07974107e-06, 5.43275186e-06, 5.40005060e-06, 5.14835951e-06,
       5.48468470e-06, 6.45800939e-06, 5.22561348e-06, 8.35742697e-06,
       6.39392486e-06, 7.32266593e-06, 1.19405239e-05, 8.84392161e-06,
       7.60803765e-06, 5.83257129e-06, 4.88489208e-06, 5.46551902e-06,
       6.09237666e-06])
Coordinates:
  * epoch     (epoch) int64 0 1 2 3 4 5 6 7 ... 269 270 271 272 273 274 275 276
    quantile  float64 0.5

## 1b) Let the *Upper Quantile Range* be defined by $Q75 - Q50$
$ UQR^s = X^{{\sigma}_T{Q75}^s} - X^{{\sigma}_T{Q50}^s}$

In [ ]:
# step B. Define the threshold
uqr = q75 - q50
uqr  # Q70 - Q50

<xarray.DataArray (epoch: 277)>
array([1.65458610e-06, 1.27921763e-06, 1.04112766e-06, 1.24351127e-06,
       1.29960321e-06, 1.14187544e-06, 1.09888199e-06, 1.70965476e-06,
       1.45130705e-06, 1.56004353e-06, 1.50997649e-06, 1.04015573e-06,
       1.29327140e-06, 1.33723461e-06, 3.13111693e-06, 8.19893799e-07,
       1.24966527e-06, 1.05442212e-06, 8.86930755e-07, 1.30006521e-06,
       6.52274883e-07, 1.48071580e-06, 1.01167220e-06, 1.37239857e-06,
       1.47166316e-06, 1.26024352e-06, 1.33107056e-06, 9.42235690e-07,
       1.62515016e-06, 1.18182776e-06, 2.01681320e-06, 1.32756148e-06,
       1.41640170e-06, 1.22148577e-06, 1.05495933e-06, 1.50257257e-06,
       1.43027735e-06, 1.63276171e-06, 1.43407602e-06, 1.31775124e-06,
       1.15011688e-06, 1.78569635e-06, 1.09602774e-06, 2.04269580e-06,
       9.23288247e-07, 1.85341242e-06, 9.70599533e-07, 1.12560689e-06,
       1.57095439e-06, 1.42370666e-06, 1.77384961e-06, 1.13306327e-06,
       1.22188484e-06, 1.35239075e-06, 1.83103844e-06, 1.54962784e-06,
       2.61460869e-06, 1.62844258e-06, 2.04703045e-06, 2.09217555e-06,
       1.57287483e-06, 1.17318278e-06, 1.86690645e-06, 1.32850600e-06,
       1.27804781e-06, 1.25466149e-06, 1.19740092e-06, 2.02023593e-06,
       1.87925550e-06, 2.33699109e-06, 2.12886012e-06, 1.25665937e-06,
       1.91857859e-06, 1.25372181e-06, 1.01598158e-06, 1.63334972e-06,
       1.33846598e-06, 1.95530537e-06, 1.59349001e-06, 1.55668208e-06,
...
       2.69443288e-06, 2.96571972e-06, 1.64629310e-06, 1.56105781e-06,
       1.87714414e-06, 2.12832092e-06, 1.69576849e-06, 1.96622485e-06,
       1.58464735e-06, 2.24644993e-06, 1.60648112e-06, 2.34716529e-06,
       2.50611383e-06, 2.23622380e-06, 2.71598563e-06, 2.41312887e-06,
       1.70676880e-06, 1.76650174e-06, 2.47546610e-06, 2.65146732e-06,
       4.76754125e-06, 2.06557607e-06, 1.33731929e-06, 1.45082310e-06,
       1.37712522e-06, 1.58409954e-06, 1.85696397e-06, 1.72330595e-06,
       1.98593010e-06, 1.87602445e-06, 2.11393201e-06, 1.77411356e-06,
       2.13465951e-06, 2.07632694e-06, 1.65730827e-06, 1.80605096e-06,
       3.01203170e-06, 1.74310760e-06, 1.19262110e-06, 1.71654618e-06,
       1.28660893e-06, 8.63169573e-07, 1.16865789e-06, 1.84491095e-06,
       8.91487807e-07, 9.84123824e-07, 1.44091295e-06, 2.26761726e-06,
       1.25195364e-06, 1.01936420e-06, 1.43695777e-06, 1.42504748e-06,
       1.41569901e-06, 1.81953790e-06, 1.30679175e-06, 1.97559989e-06,
       1.02731683e-06, 2.30026966e-06, 8.52940604e-07, 1.17715086e-06,
       8.87468484e-07, 6.56300118e-07, 1.44960585e-06, 2.62405041e-06,
       2.20025456e-06, 1.41635184e-06, 1.73879235e-06, 1.95471427e-06,
       1.44927542e-06, 1.47135343e-06, 3.83635657e-06, 1.64584465e-06,
       1.00571542e-06, 1.19091131e-06, 1.44908512e-06, 1.11587687e-06,
       1.21627091e-06])
Coordinates:
  * epoch    (epoch) int64 0 1 2 3 4 5 6 7 8 ... 269 270 271 272 273 274 275 276

---------------

### Identify outlier Indices $(i, j)$

We multiply a constant $k$ by the $UQR$ to define a measure for the spread of the right tail of the distribution of $X^{\sigma_{t}}$ values and add it to the median of $X^{\sigma_{t}}$ to obtain epoch-specific standard deviation threshold for outliers:

$$
    \tau^s_j = X^{{\sigma}_T{Q50}^S} + UQR^s\times k
$$

That is, $\tau^s_j$ is the epoch-specific threshold for the epoch $j$.

In [ ]:
# Step 1b part 2
k = 3
u_out =  q50 + q75 * k
u_out  # epoch specific thresholds

------------

Now, we compare our 2D standard deviation matrix to the threshold vector of $\tau^e_j$:

$$
X^{\sigma_{t}} \big|_{e=j}  > \tau^s_j
$$

resulting in the indicator matrix $C \in \{0, 1\}^{S \times E}=\{c_{ij}\}$ with

$$
c_{ij} =
\begin{cases}
0 & \text{if } x^{\sigma_{t}}_{ij} < \tau^s_j \\
1 & \text{if } x^{\sigma_{t}}_{ij} \geq \tau^s_j
\end{cases}
$$

Each element of this matrix indicates whether sensor $i$ is an outlier at an epoch $j$.


In [ ]:
# Step D
outlier_mask = trim_ch_sd > u_out
outlier_mask


<xarray.DataArray (epoch: 277, ch: 59)>
array([[ True,  True,  True, ..., False, False, False],
       [ True, False, False, ..., False, False, False],
       [ True,  True,  True, ..., False, False, False],
       ...,
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False]])
Coordinates:
  * epoch     (epoch) int64 0 1 2 3 4 5 6 7 ... 269 270 271 272 273 274 275 276
  * ch        (ch) <U7 'EEG 001' 'EEG 002' 'EEG 003' ... 'EEG 059' 'EEG 060'
    quantile  float64 0.5

--------------

## 1c) Identify noisy sensors part 1
To identify outlier **sensors**, we average across the **epoch** dimension of our indicator matrix $C$ and obtain $C^{\mu_e} \in \mathbb{R}^{S_\mathcal{G}}$, which is a vector of fractional numbers $c^{\mu_e}_i$ representing the percentage of **epochs** for which that sensor is an outlier.


In [ ]:
prop_outliers = outlier_mask.astype(float).mean('epoch')
prop_outliers = prop_outliers.drop_vars('quantile')
prop_outliers # porportion of epochs that sensor is an outlier

<xarray.DataArray (ch: 59)>
array([0.50541516, 0.23826715, 0.24909747, 0.62093863, 0.0866426 ,
       0.18772563, 0.70758123, 0.2166065 , 0.02166065, 0.01805054,
       0.        , 0.        , 0.        , 0.00722022, 0.6101083 ,
       0.02888087, 0.        , 0.14440433, 0.        , 0.        ,
       0.        , 0.        , 0.14801444, 0.01083032, 0.02166065,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.00361011, 0.00722022,
       0.01083032, 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.00722022, 0.        ,
       0.01083032, 0.        , 0.        , 0.        , 0.        ,
       0.03249097, 0.        , 0.00722022, 0.        , 0.00722022,
       0.00361011, 0.        , 0.        , 0.        ])
Coordinates:
  * ch       (ch) <U7 'EEG 001' 'EEG 002' 'EEG 003' ... 'EEG 059' 'EEG 060'

## 1d) Identify noisy sensors part 2

Next, we define a fractional threshold $\tau^{p}$ ($p$ for percentile; default, ``.20``) as a cutoff point for determining if a sensor should be marked artifactual. The sensor $i$ is flagged as noisy if $c^{\mu_e}_i > \tau^{p}$.


In [ ]:
threshold = 0.2
prop_outliers[prop_outliers > threshold] # FLAGGED SENSORS

<xarray.DataArray (ch: 7)>
array([0.50541516, 0.23826715, 0.24909747, 0.62093863, 0.70758123,
       0.2166065 , 0.6101083 ])
Coordinates:
  * ch       (ch) <U7 'EEG 001' 'EEG 002' 'EEG 003' ... 'EEG 008' 'EEG 015'

In [ ]:
# Let's add the outlier channels to our flags
bad_chs = prop_outliers[prop_outliers > threshold].coords.to_index().values # FLAGGED SENSORS
pipeline.flags["ch"].add_flag_cat(kind='ch_sd',
                                  bad_ch_names=bad_chs)
print(pipeline.flags['ch'])

{'ch_sd': array(['EEG 001', 'EEG 002', 'EEG 003', 'EEG 004', 'EEG 007', 'EEG 008',
       'EEG 015'], dtype=object)}


---------------

# Step 2: Flag Noisy Epochs

#### This step closely resembles the "Flag Noisy Channels" step. For the sake of brevity we will be more concise in the documentation.

<img src="https://raw.githubusercontent.com/scott-huberty/wip_pipeline-figures/main/Flag_noisy_epochs.png" alt="Flag noisy epochs" width="50%" height="50%">

--------



----------------------------------------------------------------------------


## step 2a) standard deviation across the samples dimension $T$


### **Take a moment below to notice** that the sensors flagged in step 1 are not in ``epochs_xr``


In [ ]:
epochs = pipeline.get_epochs()
# Let's make our epochs array into a named Array
epochs_xr = epochs_to_xr(epochs, kind='ch')
trim_ch_sd = epochs_xr.std("time")
trim_ch_sd

🧹 Epoching..
Not setting metadata
277 matching events found
No baseline correction applied
Created an SSP operator (subspace dimension = 3)
3 projection items activated
Using data from preloaded Raw for 277 events and 602 original time points ...
0 bad epochs dropped
Removing projector <Projection | PCA-v1, active : True, n_channels : 102>
Removing projector <Projection | PCA-v2, active : True, n_channels : 102>
Removing projector <Projection | PCA-v3, active : True, n_channels : 102>
🔍 Detecting channels to leave out of reference.
EEG channel type selected for re-referencing
Applying a custom ('EEG',) reference.


<xarray.DataArray (epoch: 277, ch: 52)>
array([[1.41267649e-05, 1.70982787e-05, 1.03874770e-05, ...,
        8.44785959e-06, 8.77453572e-06, 8.59694495e-06],
       [6.53724202e-06, 7.05730481e-06, 5.96562565e-06, ...,
        3.92898907e-06, 3.89159366e-06, 5.30487573e-06],
       [1.09075567e-05, 1.42693681e-05, 7.36062821e-06, ...,
        5.13387650e-06, 5.74995582e-06, 5.44379145e-06],
       ...,
       [7.07505011e-06, 5.08972085e-06, 7.79534265e-06, ...,
        4.17119162e-06, 4.56401918e-06, 4.92257220e-06],
       [7.59450623e-06, 7.12666225e-06, 7.78772600e-06, ...,
        4.69939303e-06, 6.25081527e-06, 5.25208140e-06],
       [6.47200903e-06, 8.52839388e-06, 7.48445513e-06, ...,
        5.23022254e-06, 5.50158684e-06, 6.07165121e-06]])
Coordinates:
  * epoch    (epoch) int64 0 1 2 3 4 5 6 7 8 ... 269 270 271 272 273 274 275 276
  * ch       (ch) <U7 'EEG 005' 'EEG 006' 'EEG 009' ... 'EEG 059' 'EEG 060'

----------

## Step 2a) part 2: We compute $\sigma^{X^{\sigma_{t}}[50,70]e}$




 50th and 75th quantile across epochs and the UQR

<br>

Like before, We Take the median and 70th quantile, but now we operate across **epochs**, resulting in two 1D vector's of size ``n_good_sensors`` $S_\mathcal{G}$

<br>

- $ X^{{\sigma}_t{Q50^e}} = Q50^e(X^{\sigma_{t}}) \in \mathbb{R}^{S_\mathcal{G}} $
- $ X^{{\sigma}_t{Q75^e}} = Q75^e(X^{\sigma_{t}}) \in \mathbb{R}^{S_\mathcal{G}} $

<br>

$ UQR^e = (X^{{\sigma}_T{Q75}^e} - X^{{\sigma}_T{Q50}^e})$

<br>

--------------------------------

## Step 2b: define sensor-specific thresholds for rejecting epochs $\tau^e_i$

Our sensor-specifc threshold for rejecting epochs is defined by:


$ \tau^e_i = X^{{\sigma}_T{Q50}^e} + UQR^e\times k$



In [ ]:
# This is step 2a
mid_val, upper_val = trim_ch_sd.quantile([.5, .75], dim='epoch')
upper_dist = upper_val - mid_val
upper_dist

<xarray.DataArray (ch: 52)>
array([2.52632100e-06, 3.53073633e-06, 8.16637185e-07, 1.47832366e-06,
       1.51709583e-06, 1.73956302e-06, 1.82628584e-06, 2.09401356e-06,
       1.91126226e-06, 9.01751728e-07, 9.07746503e-07, 5.79278250e-07,
       7.57399573e-07, 1.01483013e-06, 8.70354335e-07, 6.79759820e-07,
       8.23462286e-07, 1.07274605e-06, 4.57756588e-07, 4.40326282e-07,
       5.43342918e-07, 5.66901730e-07, 8.26022182e-07, 7.17477545e-07,
       6.36755963e-07, 5.84870815e-07, 5.56701378e-07, 9.68801136e-07,
       1.25003153e-06, 7.26190282e-07, 7.85870975e-07, 1.02182774e-06,
       8.89484081e-07, 8.02324137e-07, 6.11434350e-07, 9.10764239e-07,
       1.39453571e-06, 1.30615385e-06, 1.58028795e-06, 1.58990528e-06,
       1.69999151e-06, 1.43919111e-06, 9.21279381e-07, 1.35660382e-06,
       9.20380935e-07, 1.71887961e-06, 1.34608733e-06, 1.18013343e-06,
       1.64459808e-06, 1.41797785e-06, 1.34896420e-06, 9.89766368e-07])
Coordinates:
  * ch       (ch) <U7 'EEG 005' 'EEG 006' 'EEG 009' ... 'EEG 059' 'EEG 060'

In [ ]:
# This is step 2b
k = 3
u_out =  mid_val + upper_dist * k # sensor-specific thresholds

--------

## Step 2c) Identify Noisy Epochs part 1

<br>

#### The indicator matrix is defined by

$$
c_{ij} =
\begin{cases}
0 & \text{if } x^{\sigma_{t}}_{ij} < \tau^e_i \\
1 & \text{if } x^{\sigma_{t}}_{ij} \geq \tau^e_i
\end{cases}
$$

To identify outlier **epochs**, we average across the **sensor** dimension of our indicator matrix $C$ and obtain $C^{\mu_s} \in \mathbb{R}^{E_\mathcal{G}}$, which is a vector of fractional numbers $c^{\mu_s}_j$ representing the percentage of **sensors** for which that epoch is an outlier.


In [ ]:
# This is indicator matrix C
outlier_mask = trim_ch_sd > u_out
outlier_mask

<xarray.DataArray (epoch: 277, ch: 52)>
array([[False, False, False, ..., False, False,  True],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       ...,
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False]])
Coordinates:
  * epoch     (epoch) int64 0 1 2 3 4 5 6 7 ... 269 270 271 272 273 274 275 276
  * ch        (ch) <U7 'EEG 005' 'EEG 006' 'EEG 009' ... 'EEG 059' 'EEG 060'
    quantile  float64 0.5

In [ ]:
# This is Step 2c
prop_outliers = outlier_mask.astype(float).mean('ch')
prop_outliers = prop_outliers.drop_vars('quantile')
prop_outliers  # percent of sensors that are bad during each epoch

<xarray.DataArray (epoch: 277)>
array([0.01923077, 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.25      ,
       0.        , 0.01923077, 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.01923077, 0.01923077, 0.        , 0.        ,
       0.07692308, 0.        , 0.01923077, 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.13461538, 0.        ,
       0.42307692, 0.        , 0.09615385, 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.01923077, 0.09615385, 0.01923077, 0.        , 0.09615385,
       0.55769231, 0.01923077, 0.01923077, 0.        , 0.        ,
       0.01923077, 0.01923077, 0.        , 0.        , 0.        ,
       0.03846154, 0.        , 0.        , 0.01923077, 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.03846154,
       0.        , 0.03846154, 0.01923077, 0.07692308, 0.        ,
       0.        , 0.        , 0.        , 0.01923077, 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.01923077,
       0.        , 0.        , 0.        , 0.        , 0.        ,
...
       0.01923077, 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.01923077, 0.03846154, 0.        ,
       0.07692308, 0.11538462, 0.17307692, 0.        , 0.01923077,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.03846154, 0.        , 0.01923077, 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.01923077, 0.        ,
       0.        , 0.01923077, 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.01923077, 0.13461538, 0.46153846,
       0.36538462, 0.13461538, 0.32692308, 0.17307692, 0.        ,
       0.        , 0.01923077, 0.05769231, 0.11538462, 0.        ,
       0.        , 0.28846154, 0.09615385, 0.03846154, 0.        ,
       0.01923077, 0.        , 0.01923077, 0.        , 0.        ,
       0.05769231, 0.26923077, 0.23076923, 0.30769231, 0.        ,
       0.        , 0.        , 0.38461538, 0.15384615, 0.51923077,
       0.5       , 0.26923077, 0.17307692, 0.25      , 0.11538462,
       0.05769231, 0.07692308, 0.25      , 0.07692308, 0.01923077,
       0.03846154, 0.        , 0.        , 0.01923077, 0.01923077,
       0.03846154, 0.01923077, 0.15384615, 0.        , 0.        ,
       0.55769231, 0.28846154, 0.05769231, 0.        , 0.        ,
       0.01923077, 0.01923077])
Coordinates:
  * epoch    (epoch) int64 0 1 2 3 4 5 6 7 8 ... 269 270 271 272 273 274 275 276

----

## Step 2d: Identify Noisy Epochs Part 2

Next, we define a fractional threshold $\tau^{p}$ ($p$ for percentile; default, ``.20``) as a cutoff point for determining if a epoch should be marked artifactual. The epoch $j$ is flagged as noisy if $c^{\mu_s}_j > \tau^{p}$.


In [ ]:
threshold = 0.2
prop_outliers[prop_outliers > threshold] # FLAGGED EPOCHS

<xarray.DataArray (epoch: 24)>
array([0.25      , 0.42307692, 0.55769231, 0.30769231, 0.42307692,
       0.30769231, 0.57692308, 0.21153846, 0.23076923, 0.46153846,
       0.36538462, 0.32692308, 0.28846154, 0.26923077, 0.23076923,
       0.30769231, 0.38461538, 0.51923077, 0.5       , 0.26923077,
       0.25      , 0.25      , 0.55769231, 0.28846154])
Coordinates:
  * epoch    (epoch) int64 14 45 60 100 123 137 141 ... 250 251 253 257 270 271

In [ ]:
# Let's add the outlier epochs to our flags
bad_epochs = prop_outliers[prop_outliers > threshold].coords.to_index().values # FLAGGED Epochs
pipeline.flags["epoch"].add_flag_cat(kind='ch_sd',
                                  bad_epoch_inds=bad_epochs,
                                     epochs=epochs)
print(pipeline.flags['epoch'])

{'ch_sd': array([ 14,  45,  60, 100, 123, 137, 141, 167, 175, 219, 220, 222, 231,
       241, 242, 243, 247, 249, 250, 251, 253, 257, 270, 271])}


------


<br>
<br>


# **Step 3**: Find Nearest Neighbours & return Maximum Correlation


*Wheras steps 1 and 2 operated on a 2D matrix of standard deviation values, Steps 5, 6, 7, and 8 will operate on correlation coefficients. This Step describes the procedure for defining the 2D matrix of correlation coefficients.*

<br>

![Get nearest neighbours](https://raw.githubusercontent.com/scott-huberty/wip_pipeline-figures/main/Nearest_neighbors.png)


-----

In [ ]:
from pylossless.pipeline import chan_neighbour_r
# Posting full function definition for easier inspection
chan_neighbour_r??

<br>

#### **Take a moment to notice** that the flagged epochs from the previous step are dropped during epoching.

<br>

In [ ]:
# Notice that our flagged epochs are dropped.
epochs = pipeline.get_epochs()

🧹 Epoching..
Not setting metadata
277 matching events found
No baseline correction applied
Created an SSP operator (subspace dimension = 3)
3 projection items activated
Using data from preloaded Raw for 277 events and 602 original time points ...
57 bad epochs dropped
Removing projector <Projection | PCA-v1, active : True, n_channels : 102>
Removing projector <Projection | PCA-v2, active : True, n_channels : 102>
Removing projector <Projection | PCA-v3, active : True, n_channels : 102>
🔍 Detecting channels to leave out of reference.
EEG channel type selected for re-referencing
Applying a custom ('EEG',) reference.



----------------------



## Step 3, part 1: Calculate Correlation Coefficents between each Sensor and it's nearest Neighbors

<br>

- For each good sensor $i$ in $S_{\mathcal{G}}$, we select its $N$ nearest neighbors. I.e. the $N$ sensors that are closest to it.

- We call the sensor $i$ the *origin*, and its nearest neighbors $\hat{s_l}$ with $l \in \{1, 2, \ldots, N\}$

- Then, for each epoch $j$, we calculate the correlation coefficient $\rho^t_{(i,\hat{s_l}),j}$ between origin sensor $i$ and each neighbor $\hat{s_l}$ across dimension $t$ (samples):

Returning a 3D matrice of correlation coefficients

$$
\mathrm{P}^t = \{\rho^t_{(i, \hat{s_l}),j}\} \in \mathbb{R}^{S_G \times E_G \times n}
$$

Finally, we select the maximum correlation coefficient across the neighbor dimension $n$:


$$
\mathrm{P}^{t,{\text{max}}^n}= \max\limits_{\hat{s_l}}  \rho^t_{(i, \hat{s_l}),j}
$$


Returning a 2D matrix where each value at $(i, j)$ is the maximum correlation coefficient between sensor $i$ and its $N$ nearest neighbors, for epoch $j$

------------------------

In [ ]:
data_r_ch = chan_neighbour_r(epochs, nneigbr=3, method='max')
# maximum correlation out of correlations between ch and its 3 neighbors
data_r_ch

100%|██████████| 52/52 [00:07<00:00,  7.06it/s]


<xarray.DataArray (ch: 52, epoch: 220)>
array([[0.8565049 , 0.36647423, 0.81540625, ..., 0.57259749, 0.77485279,
        0.52104652],
       [0.89040532, 0.55660857, 0.84764715, ..., 0.70032163, 0.77208592,
        0.79683405],
       [0.82017106, 0.50140827, 0.74503166, ..., 0.66618341, 0.72751048,
        0.66401426],
       ...,
       [0.97658989, 0.90906568, 0.9708927 , ..., 0.90483325, 0.86583332,
        0.91581372],
       [0.96444752, 0.82703699, 0.92787395, ..., 0.74961163, 0.91531624,
        0.80810431],
       [0.90722028, 0.75372765, 0.85255312, ..., 0.66554706, 0.68750403,
        0.78163154]])
Coordinates:
  * epoch    (epoch) int64 0 1 2 3 4 5 6 7 8 ... 212 213 214 215 216 217 218 219
  * ch       (ch) <U7 'EEG 005' 'EEG 006' 'EEG 009' ... 'EEG 059' 'EEG 060'


---------------------

<br>

### This matrix $\mathrm{P}^{t,{\text{max}}^n}$  will be used in steps 5, 6, 7, and 8 below.

<br>

-------------------

# **Step 5**: Flag Bridged Sensors

<br>


<img src="https://raw.githubusercontent.com/scott-huberty/wip_pipeline-figures/main/Flag_bridged_sensors.png" width="50%" />



<br>

---------------------

## <center>Operations<center>

<br>

### 6a) Take the 25th, 50th, and 75th quantile across the epochs dimension of $\mathrm{P}^{t,{\text{max}}^n}$

#### *and calculate the Inter Quantile Range (IQR)*


$$
IQR^e = \mathrm{P}^{t,{\text{max}}^nQ75^e} - \mathrm{P}^{t,{\text{max}}^nQ25^e}
$$

For each sensor, divide the median across epochs by the IQR across epochs. bridged channels should have a high median correlation but a low IQR of the correlation. We call this measure the bridge-indicator.

$$
\mathcal{B}_s = \frac{\mathrm{P}^{t,{\text{max}}^nQ50^e}}{IQR^e}
$$

<br>

-------------------------------



### 6b) Define a bridging threshold

Take the 25th, 50th, and 75th quantile of $\mathcal{B}_s$

And calculate the Inter Quantile Range* $IQR^s$. A channel $i$ is bridged if $\mathcal{B}_i > B^{Q50^s} +k \times IQR^s$.

In [ ]:
import scipy
from functools import partial

msr = data_r_ch.median("epoch") / data_r_ch.reduce(scipy.stats.iqr, dim="epoch")
# msr is a 1D vector of size n_sensors
config_trim = 40
config_bridge_z = 6

trim = config_trim
if trim >= 1:
        trim /= 100
trim /= 2 # .20 and will be used as (.20, .20)

trim_mean = partial(scipy.stats.mstats.trimmed_mean,
                        limits=(trim, trim))
trim_std = partial(scipy.stats.mstats.trimmed_std,
                        limits=(trim, trim))

z_val = config_bridge_z  # 6
mask = (msr > msr.reduce(trim_mean, dim="ch")
        + z_val*msr.reduce(trim_std, dim="ch")) # bridged chans

bridged_ch_names = data_r_ch.ch.values[mask]
bridged_ch_names

array(['EEG 012', 'EEG 013', 'EEG 054', 'EEG 057'], dtype='<U7')

In [ ]:
# Let's add the outlier channels to our flags
bad_chs = bridged_ch_names
pipeline.flags["ch"].add_flag_cat(kind='bridge',
                                  bad_ch_names=bad_chs)
pipeline.flags['ch']

['EEG 012' 'EEG 013' 'EEG 054' 'EEG 057']


# Step 6: Flag Rank Channel

## The rank sensor $i$ is simply the sensor with the highest median (across epochs) correlation coefficient, out of the remaining set of good sensors, i.e.,

$$
i = \text{arg}\max\limits_i \rho_{j}^{t,{\text{max}}^n,median^j}
$$

In [ ]:
good_chs = [ch for ch in data_r_ch.ch.values
          if ch not in pipeline.flags["ch"].get_flagged()]
data_r_ch_good = data_r_ch.sel(ch=good_chs)

flag_ch = [str(data_r_ch_good.median("epoch")
                                     .idxmax(dim="ch")
                                     .to_numpy()
                            )]
pipeline.flags['ch'].add_flag_cat(kind='rank', bad_ch_names=flag_ch)
pipeline.flags['ch']['rank']

['EEG 011']


--------------------------------------

# Step 7: Flag low correlation Channels

- This step closely follows Step 1 except we use the lower end of the distribution to set the outliers threshold.

- We will be succinct in the documentation. Please Feel free to inspect the variables in the python code as needed:


----------------

## <center>Operations<center>

<br>

### 6a) Take the 30th and 50th quantile of $\mathrm{P}^{t,{\text{max}}^n}$ across the sensors dimension

*and calculate the Lower Quantile Range* $LQR^s$

Resulting in vectors $\mathrm{P}^{t,{\text{max}}^nQ25^s}$ and $\mathrm{P}^{t,{\text{max}}^nQ50^s}$ of size $E_\mathcal{G}$. The lower quantile range ($LQR$) is defined as:


$$
LQR^s = \mathrm{P}^{t,{\text{max}}^nQ50^s} - \mathrm{P}^{t,{\text{max}}^nQ25^s}
$$

<br>

### 6b) Define epoch-specific thresholds for rejecting sensors

$\tau^s_j$ represents our set of epoch-specific thresholds:


$$
\tau^s = \mathrm{P}^{t,{\text{max}}^nQ50^s} - LQR^s\times k
$$

Now, we compare our 2D correlation coefficient matrix $\mathrm{P}^{t,{\text{max}}^n} = \{ \rho^{t,{\text{max}}^n}_{ij} \}$ to the threshold vector $\tau^s_j$ resulting in the indicator matrix $C \in \{0, 1\}^{S \times E}=\{c_{ij}\}$ with

$$
c_{ij} =
\begin{cases}
1 & \text{if } \rho^{t,{\text{max}}^n}_{ij} \leq \tau^s_j \\
0 & \text{if } \rho^{t,{\text{max}}^n}_{ij} > \tau^s_j
\end{cases}
$$



### 6c) Identify uncorrelated sensors

To identify uncorrelated **sensors**, we average across the **epoch** dimension of our indicator matrix $C$ and obtain $C^{\mu_e} \in \mathbb{R}^{S_\mathcal{G}}$, which is a vector of fractional numbers $c^{\mu_e}_i$ representing the percentage of **epochs** for which that sensor is an outlier.


Next, we define a fractional threshold $\tau^{p}$ ($p$ for percentile; default, ``.20``) as a cutoff point for determining if a sensor should be marked artifactual. The sensor $i$ is flagged as noisy if $c^{\mu_e}_i > \tau^{p}$.

In [ ]:
# Step a
lower_val, mid_val = data_r_ch.quantile([.3, .5], dim='ch')

# step B. Define the threshold
lower_dist = mid_val - lower_val

# Step B
k = 3
l_out =  mid_val - lower_dist * k

# Step D
outlier_mask = data_r_ch < l_out

prop_outliers = outlier_mask.astype(float).mean('epoch')
prop_outliers = prop_outliers.drop_vars('quantile')

threshold = 0.2
prop_outliers[prop_outliers > threshold] # FLAGGED SENSORS

# Let's add the outlier channels to our flags
bad_chs = prop_outliers[prop_outliers > threshold].coords.to_index().values # FLAGGED SENSORS
pipeline.flags["ch"].add_flag_cat(kind='uncorrelated',
                                  bad_ch_names=bad_chs)
print(pipeline.flags['ch'])

{'ch_sd': array(['EEG 001', 'EEG 002', 'EEG 003', 'EEG 004', 'EEG 007', 'EEG 008',
       'EEG 015'], dtype=object), 'bridged': array(['EEG 012', 'EEG 013', 'EEG 054', 'EEG 057'], dtype='<U7'), 'uncorrelated': array(['EEG 016', 'EEG 017', 'EEG 018', 'EEG 019', 'EEG 022', 'EEG 023',
       'EEG 025', 'EEG 026', 'EEG 034'], dtype=object)}



----------------------------

<br>

# Step 8: Flag Uncorrelated Epochs

This step is similar to step 6 (flag uncorrelated channels) and step 2 (flag noisy epochs). For the sake of brevity we will not re-describe each step


---------------------------------

## <center>Operations<center>

<br>

### 6a) Take $LQR^e$ of $\mathrm{P}^{t,{\text{max}}^n}$


### 6b) Defined sensor-specific thresholds

We get the indicator matrix as described previously, using

$$
 \tau^e_i = \mathrm{P}^{t,{\text{max}}^nQ50^e} - LQR^e\times k
$$

and

$$
c_{ij} =
\begin{cases}
1 & \text{if } \rho^{t,{\text{max}}^n}_{ij} < \tau^e_i \\
0 & \text{if } \rho^{t,{\text{max}}^n}_{ij} \geq \tau^e_i
\end{cases}
$$


<br>

### 6c) Identify uncorrelated sensors
<br>

We define a fractional threshold as we did in the previous step and flag uncorrelated epochs $j$ if $c^{\mu_s}_j > \tau^{p}$.



In [ ]:
# Step a
lower_val, mid_val = data_r_ch.quantile([.3, .5], dim='epoch')

# step B. Define the threshold
lower_dist = mid_val - lower_val

# Step B
k = 3
l_out =  mid_val - lower_dist * k

# Step D
outlier_mask = data_r_ch < l_out

prop_outliers = outlier_mask.astype(float).mean('ch')
prop_outliers = prop_outliers.drop_vars('quantile')

threshold = 0.2
prop_outliers[prop_outliers < threshold] # FLAGGED EPOCHS

# Let's add the outlier epochs to our flags
bad_epochs = prop_outliers[prop_outliers > threshold].coords.to_index().values # FLAGGED EPOCHS
pipeline.flags["epoch"].add_flag_cat(kind='uncorrelated',
                                  bad_epoch_inds=bad_epochs,
                                     epochs=epochs)
print(pipeline.flags['ch'])

{'ch_sd': array(['EEG 001', 'EEG 002', 'EEG 003', 'EEG 004', 'EEG 007', 'EEG 008',
       'EEG 015'], dtype=object), 'bridged': array(['EEG 012', 'EEG 013', 'EEG 054', 'EEG 057'], dtype='<U7'), 'uncorrelated': array(['EEG 016', 'EEG 017', 'EEG 018', 'EEG 019', 'EEG 022', 'EEG 023',
       'EEG 025', 'EEG 026', 'EEG 034'], dtype=object)}


# Step 9: Run Initial ICA

In [ ]:
pipeline.run_ica('run1')

LOSSLESS: 🚩 run_ica.
🧹 Epoching..
Not setting metadata
277 matching events found
No baseline correction applied
Created an SSP operator (subspace dimension = 3)
3 projection items activated
Using data from preloaded Raw for 277 events and 602 original time points ...
85 bad epochs dropped
Removing projector <Projection | PCA-v1, active : True, n_channels : 102>
Removing projector <Projection | PCA-v2, active : True, n_channels : 102>
Removing projector <Projection | PCA-v3, active : True, n_channels : 102>
🔍 Detecting channels to leave out of reference.
EEG channel type selected for re-referencing
Applying a custom ('EEG',) reference.
Fitting ICA to data using 38 channels (please be patient, this may take a while)
Selecting by non-zero PCA components: 37 components
Fitting ICA took 19.2s.
LOSSLESS: 🏁 Finished run_ica after 21.49 seconds.


# Step 10: Identify outlying ICA Time courses

This step follows exactly step 1 so we will not re-describe the operations here.

There are however a few key differences:

- The input 3D matrix is ``(n_components, n_good_epochs, n_times)``: $ X \in \mathbb{R}^{C\times E_G \times{T}} $
- the default threshold $k$ is 6

In [ ]:
pipeline.flag_epoch_ic_sd1()

LOSSLESS: 🚩 flag_epoch_ic_sd1.
🧹 Epoching..
Not setting metadata
277 matching events found
No baseline correction applied
Created an SSP operator (subspace dimension = 3)
3 projection items activated
Using data from preloaded Raw for 277 events and 602 original time points ...
85 bad epochs dropped
Removing projector <Projection | PCA-v1, active : True, n_channels : 102>
Removing projector <Projection | PCA-v2, active : True, n_channels : 102>
Removing projector <Projection | PCA-v3, active : True, n_channels : 102>
🔍 Detecting channels to leave out of reference.
EEG channel type selected for re-referencing
Applying a custom ('EEG',) reference.
LOSSLESS: 🏁 Finished flag_epoch_ic_sd1 after 3.60 seconds.


# Step 11: Run Final ICA

# This is the Final ICA, excluding all flagged epochs and channels.

# The ``ICLabel`` Neural Network will be run on this ICA

In [ ]:
pipeline.run_ica('run2')

LOSSLESS: 🚩 run_ica.
🧹 Epoching..
Not setting metadata
277 matching events found
No baseline correction applied
Created an SSP operator (subspace dimension = 3)
3 projection items activated
Using data from preloaded Raw for 277 events and 602 original time points ...
85 bad epochs dropped
Removing projector <Projection | PCA-v1, active : True, n_channels : 102>
Removing projector <Projection | PCA-v2, active : True, n_channels : 102>
Removing projector <Projection | PCA-v3, active : True, n_channels : 102>
🔍 Detecting channels to leave out of reference.
EEG channel type selected for re-referencing
Applying a custom ('EEG',) reference.
Fitting ICA to data using 38 channels (please be patient, this may take a while)
Selecting by non-zero PCA components: 37 components
Computing Extended Infomax ICA
Fitting ICA took 116.6s.
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).


/content/pylossless/pylossless/flagging.py:260: RuntimeWarning: The provided Epochs instance is not filtered between 1 and 100 Hz. ICLabel was designed to classify features extracted from an EEG dataset bandpass filtered between 1 and 100 Hz (see the 'filter()' method for Raw and Epochs instances).
  mne_icalabel.label_components(epochs, ica, method="iclabel")


LOSSLESS: 🏁 Finished run_ica after 132.14 seconds.


# Appendix: Find outlying channels to leave out of Re Reference

- This robust average reference is performed before the first pipeline step and between most pipeline steps.

- It does not flag sensors but identifies outlier sensors and leaves them out of the reference

Like steps 1 and 2, this step starts with 3D matrix $X \in \mathbb{R}^{S \times E \times T}$ and takes the standard deviation across axis $T$:

<br>

<img src="https://raw.githubusercontent.com/scott-huberty/wip_pipeline-figures/main/robust_rereference.png" alt="Find outliers and leave out of rereference" width="50%" height="50%">


<br>

---------------------

## <center>Operations<center>

<br>

### 6a) Take the median, 30th, and 70th quantile across dimension channels of  $X^{\sigma_{t}} \in \mathbb{R}^{S \times E}$, and Calculate the Inter Quantile Range (IQR).

<br>


Create a non-parametric Z score

$$
\mathcal{Z}^{\sigma^{t}} = \frac{X^{\sigma^{t}Q50^s} - X^{\sigma^{t}}}{IQR^s}
$$




<br>

-------------------------------

<br>


### 6b) Define  threshold

Sensor $i$ is left out of the re-reference if

$$
\mathcal{Z}^{\sigma^{t}\mu^{e}} > \mathcal{Z}^{\sigma^{t}\mu^{e}Q50^s} + k \times IQR^s
$$

with $IQR^s = \mathcal{Z}^{\sigma^{t}\mu^{e}Q75^s}-\mathcal{Z}^{\sigma^{t}\mu^{e}Q25^s}$ and where $\mathcal{Z}^{\sigma^{t}\mu^{e}}$ is the mean of $\mathcal{Z}^{\sigma^{t}}$ across epochs.

In [ ]:
# This is step 2a
ch_dist = trim_ch_sd - trim_ch_sd.median(dim="ch")
perc_30 = trim_ch_sd.quantile(0.3, dim="ch")
perc_70 = trim_ch_sd.quantile(0.7, dim="ch")

# This is step b: non-parametric z-score
ch_dist /= perc_70 - perc_30  # shape (chans, epoch)

# This is step c
mean_ch_dist = ch_dist.mean(dim="epoch")

# Step D begins here
# find the median and 30 and 70 percentiles
# of the mean of the channel distributions
mdn = np.median(mean_ch_dist)
deviation = np.diff(np.quantile(mean_ch_dist, [0.3, 0.7]))
print(f"median: {mdn}, std: {deviation}")

# Thresholding. True sensors are left out
mean_ch_dist > mdn+6*deviation

median: 0.009137345273309862, std: [0.94236308]


<xarray.DataArray (ch: 52)>
array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False])
Coordinates:
  * ch       (ch) <U7 'EEG 005' 'EEG 006' 'EEG 009' ... 'EEG 059' 'EEG 060'

In [ ]:
# Here are any sensor names that are excluded
mean_ch_dist.ch[mean_ch_dist > mdn+6*deviation].values.tolist()

[]